**INICIO DO PROCESSO E CONFIGURAÇÃO DAS PASTAS**

In [0]:
# Importação de bibliotecas
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
import pandas as pd

# Configuração de diretórios
gold_path = "/Volumes/workspace/default/gold"
delta_table_path = "/Volumes/workspace/default/delta/final_climate_data"
delta_catalog = "/Volumes/workspace/default"
delta_table_name = "climate_data_final"

print("Configurando ambiente para criação da tabela Delta...")
print(f"Gold path: {gold_path}")
print(f"Delta path: {delta_table_path}")
print(f"Catalog: {delta_catalog}")
print(f"Table: {delta_table_name}")




**LENDO OS DADOS DA CAMADA GOLD E VISUALIZANDO**

In [0]:
def read_gold_data():
    """Lê todos os arquivos das camadas gold"""
    try:
        print("Lendo dados da camada Gold...")
        
        # Listar arquivos gold
        gold_files = dbutils.fs.ls(gold_path)
        weather_files = [f.path for f in gold_files if 'weather_gold' in f.name]
        air_quality_files = [f.path for f in gold_files if 'air_quality_gold' in f.name]
        
        print(f" Arquivos de weather encontrados: {len(weather_files)}")
        print(f" Arquivos de air quality encontrados: {len(air_quality_files)}")
        
        # Ler dados meteorológicos
        weather_dfs = []
        for file in weather_files:
            df = spark.read.parquet(file)
            weather_dfs.append(df)
        
        if weather_dfs:
            weather_df = weather_dfs[0]
            for df in weather_dfs[1:]:
                weather_df = weather_df.union(df)
            print(f"Total de registros meteorológicos: {weather_df.count()}")
        else:
            print(" Nenhum dado meteorológico encontrado")
            weather_df = None
        
        # Ler dados de qualidade do ar
        air_quality_dfs = []
        for file in air_quality_files:
            df = spark.read.parquet(file)
            air_quality_dfs.append(df)
        
        if air_quality_dfs:
            air_quality_df = air_quality_dfs[0]
            for df in air_quality_dfs[1:]:
                air_quality_df = air_quality_df.union(df)
            print(f"🌫️  Total de registros de qualidade do ar: {air_quality_df.count()}")
        else:
            print(" Nenhum dado de qualidade do ar encontrado")
            air_quality_df = None
        
        return weather_df, air_quality_df
        
    except Exception as e:
        print(f"Erro ao ler dados gold: {str(e)}")
        return None, None

# Ler os dados
weather_df, air_quality_df = read_gold_data()

# Visualizar amostra dos dados
if weather_df:
    print(" Amostra dos dados meteorológicos:")
    display(weather_df.limit(5))

if air_quality_df:
    print(" Amostra dos dados de qualidade do ar:")
    display(air_quality_df.limit(5))


**UNIFICANDO OS DADOS E SETANDO AS COLUNAS NECECSSARIAS**

In [0]:
def create_final_table(weather_df, air_quality_df):
    """Cria a tabela final unificada"""
    try:
        print("Unificando dados...")
        
        if weather_df is None or air_quality_df is None:
            print("Dados insuficientes para unificação")
            return None
        
        # Juntar os dados por timestamp e localização
        final_df = weather_df.alias("w").join(
            air_quality_df.alias("a"),
            (col("w.timestamp") == col("a.timestamp")) & 
            (col("w.location") == col("a.location")),
            "inner"
        ).select(
            col("w.timestamp"),
            col("w.date"),
            col("w.temperature_2m").alias("temperature_c"),
            col("w.relative_humidity_2m").alias("humidity_percent"),
            col("w.pressure_msl").alias("pressure_hpa"),
            col("w.precipitation").alias("precipitation_mm"),
            col("a.pm10"),
            col("a.pm2_5"),
            col("a.carbon_monoxide").alias("co_ugm3"),
            col("a.nitrogen_dioxide").alias("no2_ugm3"),
            col("w.location"),
            col("w.latitude"),
            col("w.longitude")
        )
        
        # Adicionar colunas calculadas
        final_df = final_df.withColumn(
            "temperature_f", 
            (col("temperature_c") * 9/5) + 32
        ).withColumn(
            "air_quality_index",
            when(col("pm2_5") <= 12, "Bom")
            .when(col("pm2_5") <= 35, "Moderado")
            .when(col("pm2_5") <= 55, "Ruim")
            .when(col("pm2_5") <= 150, "Muito Ruim")
            .otherwise("Perigoso")
        ).withColumn(
            "load_timestamp", 
            current_timestamp()
        )
        
        print(f"Dados unificados com sucesso!")
        print(f"Total de registros na tabela final: {final_df.count()}")
        print(f"Período dos dados: {final_df.agg(min('timestamp'), max('timestamp')).collect()[0]}")
        
        # Mostrar schema final
        print("  Schema da tabela final:")
        final_df.printSchema()
        
        return final_df
        
    except Exception as e:
        print(f"Erro na unificação dos dados: {str(e)}")
        return None

# Criar tabela final
final_table_df = create_final_table(weather_df, air_quality_df)

if final_table_df:
    print(" Amostra da tabela final:")
    display(final_table_df.limit(10))


**V2 AJUSTADA LENDO AS BIBLIOTECAS E DIRETORIOS**

In [0]:
# Importação de bibliotecas
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
import pandas as pd

# Configuração de diretórios
gold_path = "/Volumes/workspace/default/gold"
delta_table_path = "/Volumes/workspace/default/delta/final_climate_data"
delta_catalog = "/Volumes/workspace/default"
delta_table_name = "climate_data_final"

print("Configurando ambiente para criação da tabela Delta...")
print(f"Gold path: {gold_path}")
print(f"Delta path: {delta_table_path}")
print(f"Catalog: {delta_catalog}")
print(f"Table: {delta_table_name}")

**LENDO OS DADOS  DA CAMADA GOLD E VISUALIZANDO OS DADOS**

In [0]:
def read_gold_data():
    """Lê todos os arquivos das camadas gold"""
    try:
        print(" Lendo dados da camada Gold...")
        
        # Listar arquivos gold usando a sintaxe correta
        gold_files = dbutils.fs.ls(gold_path.replace("dbfs:", ""))
        weather_files = [f.path for f in gold_files if 'weather_gold' in f.name]
        air_quality_files = [f.path for f in gold_files if 'air_quality_gold' in f.name]
        
        print(f"  Arquivos de weather encontrados: {len(weather_files)}")
        print(f" Arquivos de air quality encontrados: {len(air_quality_files)}")
        
        # Ler dados meteorológicos
        weather_dfs = []
        for file in weather_files:
            df = spark.read.parquet(file)
            weather_dfs.append(df)
        
        if weather_dfs:
            weather_df = weather_dfs[0]
            for df in weather_dfs[1:]:
                weather_df = weather_df.union(df)
            print(f"  Total de registros meteorológicos: {weather_df.count()}")
        else:
            print("  Nenhum dado meteorológico encontrado")
            weather_df = None
        
        # Ler dados de qualidade do ar
        air_quality_dfs = []
        for file in air_quality_files:
            df = spark.read.parquet(file)
            air_quality_dfs.append(df)
        
        if air_quality_dfs:
            air_quality_df = air_quality_dfs[0]
            for df in air_quality_dfs[1:]:
                air_quality_df = air_quality_df.union(df)
            print(f"🌫️  Total de registros de qualidade do ar: {air_quality_df.count()}")
        else:
            print("  Nenhum dado de qualidade do ar encontrado")
            air_quality_df = None
        
        return weather_df, air_quality_df
        
    except Exception as e:
        print(f" Erro ao ler dados gold: {str(e)}")
        return None, None

# Ler os dados
weather_df, air_quality_df = read_gold_data()

# Visualizar amostra dos dados
if weather_df:
    print(" Amostra dos dados meteorológicos:")
    display(weather_df.limit(5))

if air_quality_df:
    print(" Amostra dos dados de qualidade do ar:")
    display(air_quality_df.limit(5))


****CRIAÇÃO DA TABELA  APLICANDO O JOIN** **

In [0]:

def create_final_table(weather_df, air_quality_df):
    """Cria a tabela final unificada"""
    try:
        print("Unificando dados...")
        
        if weather_df is None or air_quality_df is None:
            print(" Dados insuficientes para unificação")
            return None
        
        # Juntar os dados por timestamp e localização 
        final_df = weather_df.alias("w").join(
            air_quality_df.alias("a"),
            (col("w.timestamp") == col("a.timestamp")) & 
            (col("w.location") == col("a.location")),
            "inner"
        ).select(
            col("w.timestamp"),
            col("w.date"),
            col("w.temperature_2m").alias("temperature_c"),
            col("w.relative_humidity_2m").alias("humidity_percent"),
            col("w.pressure_msl").alias("pressure_hpa"),
            col("w.precipitation").alias("precipitation_mm"),
            col("a.pm10"),
            col("a.pm2_5"),
            col("a.carbon_monoxide").alias("co_ugm3"),
            col("a.nitrogen_dioxide").alias("no2_ugm3"),
            col("w.location"),
            col("w.latitude"),
            col("w.longitude")
           
        )
        
        # Adicionar colunas calculadas 
        final_df = final_df.withColumn(  
            "temperature_f", 
            (col("temperature_c") * 9/5) + 32
        ).withColumn(
            "air_quality_index",
            when(col("pm2_5") <= 12, "Bom")
            .when(col("pm2_5") <= 35, "Moderado")
            .when(col("pm2_5") <= 55, "Ruim")
            .when(col("pm2_5") <= 150, "Muito Ruim")
            .otherwise("Perigoso")
        ).withColumn(
            "load_timestamp", 
            current_timestamp()
        )
        
        print(f" Dados unificados com sucesso!")
        print(f"Total de registros na tabela final: {final_df.count()}")
        
        # Mostrar schema final
        print(" Schema da tabela final:")
        final_df.printSchema()
        
        return final_df
        
    except Exception as e:
        print(f" Erro na unificação dos dados: {str(e)}")
        return None

# Criar tabela final
final_table_df = create_final_table(weather_df, air_quality_df)

if final_table_df:
    print("Amostra da tabela final:")
    display(final_table_df.limit(10))


**CRIANDO  TABELA FINAL USANDO SQL**

In [0]:
def create_delta_table(final_df):
    """Cria a tabela Delta final """
    try:
        print(" Criando tabela Delta...")
        
        if final_df is None:
            print(" Nenhum dado para criar tabela")
            return None
        
        #  Unity Catalog
        
        catalog_name = "workspace"  #  catalog
        schema_name = "default"     #  schema
        table_name = "climate_data_final"
        full_table_name = f"{catalog_name}.{schema_name}.{table_name}"
        
        
        
        
        # Criar DataFrame temporário
        final_df.createOrReplaceTempView("temp_final_table")
        
        # Criar tabela usando SQL 
        spark.sql(f"""
            CREATE OR REPLACE TABLE {full_table_name}
            USING DELTA
            AS SELECT * FROM temp_final_table
        """)
        
        print(f" Tabela Delta criada com sucesso!")
        print(f"Tabela: {full_table_name}")
        
        # Verificar a tabela criada
        delta_df = spark.read.table(full_table_name)
        print(f"Total de registros na tabela Delta: {delta_df.count()}")
        
        return delta_df
        
    except Exception as e:
        print(f"Erro ao criar tabela Delta: {str(e)}")
        print(" Tentando método alternativo...")
        
        # Método alternativo usando DataFrameWriter
        try:
            # Escrever usando DataFrameWriter
            (final_df.write
                .format("delta")
                .mode("overwrite")
                .option("overwriteSchema", "true")
                .saveAsTable(full_table_name))
            
            print(f" Tabela Delta criada com método alternativo!")
            delta_df = spark.read.table(full_table_name)
            return delta_df
            
        except Exception as e2:
            print(f" Erro no método alternativo: {str(e2)}")
            return None


**CRIANDO A TABELA DELTA**

In [0]:
# Configuração 
catalog_name = "workspace"  # Seu catalog
schema_name = "default"     # Seu schema
table_name = "climate_data_final"
full_table_name = f"{catalog_name}.{schema_name}.{table_name}"

print("Configuração para Unity Catalog:")
print(f"Catalog: {catalog_name}")
print(f" Schema: {schema_name}")
print(f"Table: {table_name}")
print(f" Full name: {full_table_name}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 4.2 Executar Criação da Tabela

# COMMAND ----------

# Criar tabela Delta
delta_final_df = create_delta_table(final_table_df)

if delta_final_df:
    print(" Amostra da tabela Delta:")
    display(delta_final_df.limit(10))
else:
    print("Falha ao criar tabela Delta")


**CHECKLIST PARA VERIFICAÇÃO DA CRIAÇÃO DA TABELA**

In [0]:
# Verificar se a tabela foi criada corretamente
try:
    # Listar tabelas no schema
    tables = spark.sql(f"SHOW TABLES IN {catalog_name}.{schema_name} LIKE '{table_name}'")
    if tables.count() > 0:
        print("Tabela encontrada no Unity Catalog!")
        display(tables)
        
        # Detalhes da tabela
        table_details = spark.sql(f"DESCRIBE DETAIL {full_table_name}")
        print("Detalhes da tabela:")
        display(table_details)
        
        # Contagem de registros
        count = spark.sql(f"SELECT COUNT(*) FROM {full_table_name}").collect()[0][0]
        print(f" Total de registros: {count:,}")
        
    else:
        print(" Tabela não encontrada no Unity Catalog")
        
except Exception as e:
    print(f" Erro ao verificar tabela: {str(e)}")


**QUERYS SQL PARA CONSULTA DOS DADOS**

In [0]:
# consulta na tabela
try:
    print("Testando consulta na tabela...")
    test_query = spark.sql(f"""
        SELECT 
            timestamp,
            temperature_c,
            humidity_percent,
            pm2_5,
            air_quality_index
        FROM {full_table_name}
        ORDER BY timestamp DESC
        LIMIT 5
    """)
    display(test_query)
    print("Consulta executada com sucesso!")
    
except Exception as e:
    print(f"Erro na consulta: {str(e)}")
    print(" Tentando ver tabelas disponíveis...")
    
    # Listar todas as tabelas disponíveis
    try:
        all_tables = spark.sql("SHOW TABLES")
        display(all_tables)
    except:
        print("Não foi possível listar tabelas")


In [0]:
# SQL : Dados mais recentes
display(
    spark.sql(
        """
        SELECT
            timestamp,
            temperature_c,
            humidity_percent,
            pm2_5,
            air_quality_index
        FROM climate_data_final
        ORDER BY timestamp DESC
        LIMIT 10
        """
    )
)

In [0]:
# SQL: Médias por hora
print("Médias por hora:")
display(spark.sql(f"""
    SELECT
        DATE_TRUNC('hour', timestamp) as hour,
        ROUND(AVG(temperature_c), 1) as avg_temperature,
        ROUND(AVG(pm2_5), 1) as avg_pm2_5,
        COUNT(*) as readings
    FROM climate_data_final
    GROUP BY DATE_TRUNC('hour', timestamp)
    ORDER BY hour DESC
    LIMIT 12
"""))


In [0]:

# SQL: Qualidade do ar por faixa
print("Qualidade do ar por faixa:")
display(spark.sql(f"""
    SELECT
        air_quality_index,
        MIN(pm2_5) as min_pm2_5,
        MAX(pm2_5) as max_pm2_5,
        AVG(pm2_5) as avg_pm2_5,
        COUNT(*) as readings
    FROM climate_data_final
    GROUP BY air_quality_index
    ORDER BY avg_pm2_5
"""))